### Setup

In [1]:
import os
import pandas as pd

dataset_input = "Datasets/"

### Utilities

In [2]:
def read_directory(path, extension='.csv'):
    return [f for f in os.listdir(path) if f.endswith(extension)]

def write_csv(path, dataset):
    dataset.to_csv(path, index=False)

def read_data(path, columns_to_keep=[]):
    files = read_directory(path)
    dfs = []
    keys = []
    for file in files:
        file_path = os.path.join(path, file)
        year = str(file).split('.')[0].split('_')[2]
        
        df = pd.read_csv(file_path)
        df.fillna(0, inplace=True)
        filtered_df = df[columns_to_keep]
        
        keys.append(year)
        dfs.append(filtered_df)

    result = pd.concat(dfs, keys=keys, axis=0)
    result.reset_index(level=0, inplace=True)
    result.rename(columns={'level_0': 'season'}, inplace=True)
    result.reset_index(drop=True, inplace=True)
    return result

### Preprocessing

In [3]:
df_ratings = read_data(
        path=os.path.join(dataset_input, 'ratings'),
        columns_to_keep=[
            'sofifa_id',
            'attacking_finishing',
            'attacking_heading_accuracy',
            'attacking_short_passing',
            'attacking_volleys',
            'defending_marking',
            'defending_sliding_tackle',
            'defending_standing_tackle',
            'gk_diving',
            'gk_handling',
            'gk_positioning',
            'gk_reflexes',
            'gk_speed',
            'mentality_interceptions',
            'power_long_shots',
            'skill_long_passing'
        ])

df_ratings.head()

,season,sofifa_id,attacking_finishing,attacking_heading_accuracy,attacking_short_passing,attacking_volleys,defending_marking,defending_sliding_tackle,defending_standing_tackle,gk_diving,gk_handling,gk_positioning,gk_reflexes,gk_speed,mentality_interceptions,power_long_shots,skill_long_passing
0,20162017,20801,93,87,82,88,22.0,23,31,0.0,0.0,0.0,0.0,0.0,29,92,72
1,20162017,158023,95,71,88,85,13.0,26,28,0.0,0.0,0.0,0.0,0.0,22,88,87
2,20162017,190871,89,62,81,83,21.0,33,24,0.0,0.0,0.0,0.0,0.0,36,77,75
3,20162017,167495,13,25,55,11,10.0,11,10,89.0,90.0,91.0,89.0,60.0,30,16,59
4,20162017,176580,94,77,83,88,30.0,38,45,0.0,0.0,0.0,0.0,0.0,41,86,64


In [4]:
df_matches = read_data(
        path=os.path.join(dataset_input, 'matches'),
        columns_to_keep=[
            'match_url', 
            'home_formation', 
            'home_team', 
            'home_goals', 
            'away_goals', 
            'away_team', 
            'away_formation', 
            'home_sequence', 
            'away_sequence', 
            'home_xi_sofifa_ids', 
            'away_xi_sofifa_ids'
        ])

df_matches['match_id'] = df_matches['match_url'].apply(lambda x: str(x).split('/')[-1])
df_matches = df_matches[['match_id'] + df_matches.columns[:-1].to_numpy().tolist()]
df_matches = df_matches.drop(columns=['match_url'])
df_matches

,match_id,season,home_formation,home_team,home_goals,away_goals,away_team,away_formation,home_sequence,away_sequence,home_xi_sofifa_ids,away_xi_sofifa_ids
0,12313,20152016,4-4-2,Crystal Palace,0,3,Chelsea,4-2-3-1,"C,R,CR,CL,L,R,CR,CL,L,CR,CL","C,R,CR,CL,L,CR,CL,RL,C,LR,C","170084.0,186392.0,157665.0,159577.0,193475.0,2...","192119.0,178372.0,204311.0,13732.0,184432.0,16..."
1,12284,20152016,4-2-3-1,Arsenal,2,1,Manchester City,4-4-1-1,"C,R,CR,CL,L,CR,CL,RL,C,LR,C","C,R,CR,CL,L,R,CR,CL,L,C,C","48940.0,203747.0,53612.0,165229.0,177604.0,156...","150724.0,163419.0,192366.0,190531.0,185103.0,1..."
2,12346,20152016,4-2-3-1,Crystal Palace,1,2,AFC Bournemouth,4-1-4-1,"C,R,CR,CL,L,CR,CL,RL,C,LR,C","C,R,CR,CL,L,C,R,CR,CL,L,C","170084.0,186392.0,157665.0,159577.0,193475.0,1...","105846.0,190885.0,135883.0,193011.0,169638.0,1..."
3,12141,20152016,4-2-3-1,Everton,0,2,Manchester City,4-2-3-1,"C,R,CR,CL,L,CR,CL,RL,C,LR,C","C,R,CR,CL,L,CR,CL,RL,C,LR,C","16254.0,180216.0,203574.0,53914.0,209852.0,188...","150724.0,163419.0,139720.0,190531.0,185103.0,1..."
4,12232,20152016,4-2-3-1,Arsenal,1,1,Tottenham Hotspur,4-2-3-1,"C,R,CR,CL,L,CR,CL,RL,C,LR,C","C,R,CR,CL,L,CR,CL,RL,C,LR,C","48940.0,158626.0,53612.0,165229.0,177604.0,146...","167948.0,188377.0,184087.0,172871.0,169595.0,2..."
...,...,...,...,...,...,...,...,...,...,...,...,...
1515,22417,20172018,4-3-3,Manchester City,7,2,Stoke City,4-4-1-1,"C,R,CR,CL,L,RL,C,LR,RL,C,LR","C,R,CR,CL,L,R,CR,CL,L,C,C","210257.0,188377.0,203574.0,192366.0,183427.0,1...","203042.0,238319.0,204311.0,209997.0,178567.0,1..."
1516,22527,20172018,4-3-3,Manchester City,4,0,AFC Bournemouth,3-4-2-1,"C,R,CR,CL,L,RL,C,LR,RL,C,LR","C,RL,C,LR,R,CR,CL,L,CR,CL,C","210257.0,188377.0,139720.0,192366.0,183427.0,1...","172723.0,193011.0,208920.0,230882.0,190885.0,1..."
1517,22352,20172018,4-4-1-1,AFC Bournemouth,0,2,Watford,4-3-3,"C,R,CR,CL,L,R,CR,CL,L,C,C","C,R,CR,CL,L,RL,C,LR,RL,C,LR","172723.0,190885.0,193011.0,208920.0,169638.0,2...","135451.0,189860.0,177578.0,189067.0,182744.0,1..."
1518,22623,20172018,4-4-1-1,Burnley,2,1,Everton,4-3-3,"C,R,CR,CL,L,R,CR,CL,L,C,C","C,R,CR,CL,L,RL,C,LR,RL,C,LR","203841.0,199633.0,202695.0,189682.0,165321.0,1...","204935.0,180216.0,163824.0,207599.0,204519.0,2..."


In [30]:
# preprocess team formation, team with shorter formation length will be stretched defensively for away and offensively for home
# eg: away: 4-4-2, home: 4-1-2-1-2
# match position grid =>                    [D, M0, M1, M2, F], based on wider home team formation
# away player will be mapped offensively to [D, M0,         F]
# NOTE: how to handle ball passing from D to M0?
def process_formation(row):
    home_formation     = row['home_formation'].split('-')
    away_formation     = row['away_formation'].split('-')
    home_formation_len = len(home_formation)
    away_formation_len = len(away_formation)
    formation_len_diff = abs(away_formation_len - home_formation_len)
    max_formation_len  = max(away_formation_len, home_formation_len)
    formation_grid = ['D'] + ['M' + str(i) for i in range(0, max_formation_len - 2)] + ['F']

    if formation_len_diff == 0:
        # equal len
        home_formation_position = ','.join([x for x, y in zip(formation_grid, home_formation) for _ in range(int(y))])
        away_formation_position = ','.join([x for x, y in zip(formation_grid, away_formation) for _ in range(int(y))])
    else:
        if away_formation_len > home_formation_len:
            away_formation_position = ','.join([x for x, y in zip(formation_grid, away_formation) for _ in range(int(y))])
            # stretch home defensively
            expanded_formation = home_formation[:1] + ['0' * formation_len_diff] + home_formation[1:]
            home_formation_position = ','.join([x for x, y in zip(formation_grid, expanded_formation) for _ in range(int(y))])
        else:
            home_formation_position = ','.join([x for x, y in zip(formation_grid, home_formation) for _ in range(int(y))])
            # stretch away aggressively
            expanded_formation = away_formation[:-1] + ['0' * formation_len_diff] + away_formation[-1:]
            away_formation_position = ','.join([x for x, y in zip(formation_grid, expanded_formation) for _ in range(int(y))])

    row['formation_grid'] = formation_grid
    row['home_formation_position'] = 'K,' + home_formation_position
    row['away_formation_position'] = 'K,' + away_formation_position
    return row

df_matches_processed = df_matches.copy()
df_matches_processed[['home_formation', 'away_formation', 'formation_grid', 'home_formation_position', 'away_formation_position']] = df_matches[['home_formation', 'away_formation']].apply(process_formation, axis=1)
df_matches_processed[df_matches_processed['match_id'] == '12313']

,match_id,season,home_formation,home_team,home_goals,away_goals,away_team,away_formation,home_sequence,away_sequence,home_xi_sofifa_ids,away_xi_sofifa_ids,formation_grid,home_formation_position,away_formation_position
0,12313,20152016,4-4-2,Crystal Palace,0,3,Chelsea,4-2-3-1,"C,R,CR,CL,L,R,CR,CL,L,CR,CL","C,R,CR,CL,L,CR,CL,RL,C,LR,C","170084.0,186392.0,157665.0,159577.0,193475.0,2...","192119.0,178372.0,204311.0,13732.0,184432.0,16...","[D, M0, M1, F]","K,D,D,D,D,M1,M1,M1,M1,F,F","K,D,D,D,D,M0,M0,M1,M1,M1,F"


In [33]:
# explode 'home_sequence', 'away_sequence', 'home_xi_sofifa_ids', 'away_xi_sofifa_ids'
df_matches_exploded = df_matches_processed.copy()
df_matches_exploded['home_sequence']            = df_matches_exploded['home_sequence'].str.split(',')
df_matches_exploded['away_sequence']            = df_matches_exploded['away_sequence'].str.split(',')
df_matches_exploded['home_xi_sofifa_ids']       = df_matches_exploded['home_xi_sofifa_ids'].str.split(',')
df_matches_exploded['away_xi_sofifa_ids']       = df_matches_exploded['away_xi_sofifa_ids'].str.split(',')
df_matches_exploded['home_formation_position']  = df_matches_exploded['home_formation_position'].str.split(',')
df_matches_exploded['away_formation_position']  = df_matches_exploded['away_formation_position'].str.split(',')
df_matches_exploded['home_court']               = 'home'
df_matches_exploded['away_court']               = 'away'

df_matches_exploded = df_matches_exploded.explode(['home_sequence', 'away_sequence', 'home_xi_sofifa_ids', 'away_xi_sofifa_ids', 'home_formation_position', 'away_formation_position'])
df_matches_exploded.reset_index(drop=True, inplace=True)
df_matches_exploded[df_matches_exploded['match_id'] == '12313']


,match_id,season,home_formation,home_team,home_goals,away_goals,away_team,away_formation,home_sequence,away_sequence,home_xi_sofifa_ids,away_xi_sofifa_ids,formation_grid,home_formation_position,away_formation_position,home_court,away_court
0,12313,20152016,4-4-2,Crystal Palace,0,3,Chelsea,4-2-3-1,C,C,170084.0,192119.0,"[D, M0, M1, F]",K,K,home,away
1,12313,20152016,4-4-2,Crystal Palace,0,3,Chelsea,4-2-3-1,R,R,186392.0,178372.0,"[D, M0, M1, F]",D,D,home,away
2,12313,20152016,4-4-2,Crystal Palace,0,3,Chelsea,4-2-3-1,CR,CR,157665.0,204311.0,"[D, M0, M1, F]",D,D,home,away
3,12313,20152016,4-4-2,Crystal Palace,0,3,Chelsea,4-2-3-1,CL,CL,159577.0,13732.0,"[D, M0, M1, F]",D,D,home,away
4,12313,20152016,4-4-2,Crystal Palace,0,3,Chelsea,4-2-3-1,L,L,193475.0,184432.0,"[D, M0, M1, F]",D,D,home,away
5,12313,20152016,4-4-2,Crystal Palace,0,3,Chelsea,4-2-3-1,R,CR,212014.0,162895.0,"[D, M0, M1, F]",M1,M0,home,away
6,12313,20152016,4-4-2,Crystal Palace,0,3,Chelsea,4-2-3-1,CR,CL,181483.0,164477.0,"[D, M0, M1, F]",M1,M0,home,away
7,12313,20152016,4-4-2,Crystal Palace,0,3,Chelsea,4-2-3-1,CL,RL,171972.0,180403.0,"[D, M0, M1, F]",M1,M1,home,away
8,12313,20152016,4-4-2,Crystal Palace,0,3,Chelsea,4-2-3-1,L,C,155355.0,188152.0,"[D, M0, M1, F]",M1,M1,home,away
9,12313,20152016,4-4-2,Crystal Palace,0,3,Chelsea,4-2-3-1,CR,LR,169586.0,183277.0,"[D, M0, M1, F]",F,M1,home,away


In [34]:
# Build a realtion table
df_match_ratings = pd.DataFrame()
df_match_ratings['match_id']            = pd.concat([df_matches_exploded['match_id'],                   df_matches_exploded['match_id']])
df_match_ratings['season']              = pd.concat([df_matches_exploded['season'],                     df_matches_exploded['season']])
df_match_ratings['team']                = pd.concat([df_matches_exploded['home_team'],                  df_matches_exploded['away_team']])
df_match_ratings['court']               = pd.concat([df_matches_exploded['home_court'],                 df_matches_exploded['away_court']])
df_match_ratings['sofifa_id']           = pd.concat([df_matches_exploded['home_xi_sofifa_ids'],         df_matches_exploded['away_xi_sofifa_ids']])
df_match_ratings['formation_position']  = pd.concat([df_matches_exploded['home_formation_position'],    df_matches_exploded['away_formation_position']])
df_match_ratings['position']            = pd.concat([df_matches_exploded['home_sequence'],              df_matches_exploded['away_sequence']])

df_match_ratings.reset_index(drop=True, inplace=True)
df_match_ratings['sofifa_id'] = df_match_ratings['sofifa_id'].apply(lambda x: int(float(x)))

df_match_ratings[df_match_ratings['match_id'] == '12313']


,match_id,season,team,court,sofifa_id,formation_position,position
0,12313,20152016,Crystal Palace,home,170084,K,C
1,12313,20152016,Crystal Palace,home,186392,D,R
2,12313,20152016,Crystal Palace,home,157665,D,CR
3,12313,20152016,Crystal Palace,home,159577,D,CL
4,12313,20152016,Crystal Palace,home,193475,D,L
5,12313,20152016,Crystal Palace,home,212014,M1,R
6,12313,20152016,Crystal Palace,home,181483,M1,CR
7,12313,20152016,Crystal Palace,home,171972,M1,CL
8,12313,20152016,Crystal Palace,home,155355,M1,L
9,12313,20152016,Crystal Palace,home,169586,F,CR


In [35]:
# Join match with player ratings by sofifa_id and season
df_merged_match_rating = df_match_ratings.merge(df_ratings, on=['sofifa_id', 'season'], how='inner')
df_merged_match_rating[df_merged_match_rating['match_id'] == '12313']


,match_id,season,team,court,sofifa_id,formation_position,position,attacking_finishing,attacking_heading_accuracy,attacking_short_passing,...,defending_sliding_tackle,defending_standing_tackle,gk_diving,gk_handling,gk_positioning,gk_reflexes,gk_speed,mentality_interceptions,power_long_shots,skill_long_passing
0,12313,20152016,Crystal Palace,home,170084,K,C,19,14,12,...,13,18,72.0,74.0,76.0,72.0,45.0,22,14,13
29,12313,20152016,Crystal Palace,home,186392,D,R,46,75,77,...,78,80,0.0,0.0,0.0,0.0,0.0,77,63,72
59,12313,20152016,Crystal Palace,home,157665,D,CR,34,81,69,...,78,81,0.0,0.0,0.0,0.0,0.0,80,32,65
94,12313,20152016,Crystal Palace,home,159577,D,CL,32,78,54,...,78,76,0.0,0.0,0.0,0.0,0.0,70,59,62
126,12313,20152016,Crystal Palace,home,193475,D,L,39,77,74,...,78,76,0.0,0.0,0.0,0.0,0.0,67,54,76
160,12313,20152016,Crystal Palace,home,212014,M1,R,45,43,56,...,65,66,0.0,0.0,0.0,0.0,0.0,65,63,65
191,12313,20152016,Crystal Palace,home,181483,M1,CR,59,79,72,...,68,81,0.0,0.0,0.0,0.0,0.0,84,74,68
207,12313,20152016,Crystal Palace,home,171972,M1,CL,56,60,78,...,71,74,0.0,0.0,0.0,0.0,0.0,75,61,76
233,12313,20152016,Crystal Palace,home,155355,M1,L,62,71,73,...,43,52,0.0,0.0,0.0,0.0,0.0,56,62,66
237,12313,20152016,Crystal Palace,home,169586,F,CR,73,68,64,...,28,35,0.0,0.0,0.0,0.0,0.0,22,62,41


### Generate PSC files
Available tables:
- df_matches_processed
   ```
   ['match_id',
   'season',
   'home_formation',
   'home_team',
   'home_goals',
   'away_goals',
   'away_team',
   'away_formation',
   'home_sequence',
   'away_sequence',
   'home_xi_sofifa_ids',
   'away_xi_sofifa_ids',
   'formation_grid',
   'home_formation_position',
   'away_formation_position']
   ```
- df_merged_match_rating
   ```
   ['match_id',
    'season',
    'team',
    'sofifa_id',
    'formation_position',
    'position',
    'attacking_finishing',
    'attacking_heading_accuracy',
    'attacking_short_passing',
    'attacking_volleys',
    'defending_marking',
    'defending_sliding_tackle',
    'defending_standing_tackle',
    'gk_diving',
    'gk_handling',
    'gk_positioning',
    'gk_reflexes',
    'gk_speed',
    'mentality_interceptions',
    'power_long_shots',
    'skill_long_passing']
   ```


#### 1. Generate state flow based on `formation_grid`
possible combos:
```
[K, D, M0, F]
[k, D, M0, M1 ,F]
[k, D, M0, M1 ,M2, F]
```

In [18]:
input_match = '12313'

In [66]:
formation_position = {
'L'  : 0,
'LR' : 1,
'CL' : 2,
'C'  : 3,
'CR' : 4,
'RL' : 5,
'R'  : 6,
}


In [67]:
formation_grid = df_matches_processed[df_matches_processed['match_id'] == input_match]['formation_grid']
home_players = df_merged_match_rating[(df_merged_match_rating['match_id'] == input_match) & (df_merged_match_rating['court'] == 'home')]
away_players = df_merged_match_rating[(df_merged_match_rating['match_id'] == input_match) & (df_merged_match_rating['court'] == 'away')]

In [112]:
home_players
((home_players['formation_position'] == 'K') & (home_players['position'] == 'C')).any()

True

In [92]:
# Generate field grid for teams
#         L, LR, CL, C, CR, RL, R
# [-1(6), 0,  0,  0, 0, 0,  0,  0, -1(6)];
df_match_field_grid = pd.DataFrame(columns=['L', 'LR', 'CL', 'C', 'CR', 'RL', 'R'])

home_field_grid = df_match_field_grid
away_field_grid = df_match_field_grid.copy()

home_field_grid['formation_position'] = formation_grid.explode()
away_field_grid['formation_position'] = formation_grid.explode()

home_field_grid = home_field_grid.fillna(0)
away_field_grid = away_field_grid.fillna(0)

In [93]:
home_field_grid

,L,LR,CL,C,CR,RL,R,formation_position
0,0,0,0,0,0,0,0,D
0,0,0,0,0,0,0,0,M0
0,0,0,0,0,0,0,0,M1
0,0,0,0,0,0,0,0,F


In [119]:
# Set player positions
def fill_players(row, players):
    for pos in formation_position:
        if (((players['formation_position'] == row['formation_position']) & (players['position'] == pos)).any()):
            row[pos] = 1
    return row

home_field_grid = home_field_grid.apply(lambda x: fill_players(x, home_players), axis=1)
away_field_grid = away_field_grid.apply(lambda x: fill_players(x, away_players), axis=1)
away_field_grid

,L,LR,CL,C,CR,RL,R,formation_position
0,1,0,1,0,1,0,1,D
0,0,0,1,0,1,0,0,M0
0,0,1,0,1,0,1,0,M1
0,0,0,0,1,0,0,0,F


In [120]:
home_field_grid

,L,LR,CL,C,CR,RL,R,formation_position
0,1,0,1,0,1,0,1,D
0,0,0,0,0,0,0,0,M0
0,1,0,1,0,1,0,1,M1
0,0,0,1,0,1,0,0,F
